In [1]:
import pandas as pd
import numpy as np
import requests

url = 'https://api.pushshift.io/reddit/search/submission'

In [2]:
def pull_request(subreddit, timestamp):
    if timestamp == 0:       
        params = {
            'subreddit': subreddit,
            'size': 100,
        }
    else:
        params = {
            'subreddit': subreddit,
            'size': 100,
            'before': timestamp
        }

    res = requests.get(url, params)

    data = res.json()
    posts = data['data']

    df_store = pd.DataFrame(posts)
    return df_store[['created_utc', 'author', 'selftext', 'title', 'subreddit']]

In [3]:
def get_all_posts(subreddit, num_pulls):
    master_df = pull_request(subreddit, 0)
    end_time = int(master_df['created_utc'][-1:])

    num_pull_requests = num_pulls

    while num_pull_requests > 1:
        try:
            df = pull_request(subreddit, end_time)
            master_df = pd.concat([master_df, df], axis=0)
            end_time = int(df['created_utc'][-1:])
            num_pull_requests -= 1
        except:
            continue
    
    return master_df

In [4]:
%time final_df = get_all_posts('GrandTheftAutoV', 200)

CPU times: user 18.5 s, sys: 977 ms, total: 19.4 s
Wall time: 3min 13s


In [5]:
final_df

,created_utc,author,selftext,title,subreddit
0,1615996307,Chchbitcheetah,I have a PS4 and looking to find another girl ...,Someone play GTA,GrandTheftAutoV
1,1615994614,MidoriTea,"I know you can do the 50 stunt jumps online, b...",Under the Bridge online?,GrandTheftAutoV
2,1615993498,reol22,,Ps3 modded accounts for sale 15 euros paypal h...,GrandTheftAutoV
3,1615992134,Litla_Tikin,Could this be because of the update? and is th...,The game is lagging so much all of a sudden...,GrandTheftAutoV
4,1615990906,DivineCheeta,"Hi, im experiencing a bug possibly in Gta Onli...",Treasure hunt issues.,GrandTheftAutoV
...,...,...,...,...,...
95,1576426886,Hayxe,[removed],Unreleased F.I.B content? Glitch? Easter egg?,GrandTheftAutoV
96,1576425576,OFF-WH1TE,**PC AND PS4 GTA CARMEETS | HYPE CAR CLUB**📷\n...,GTA V CARMEETS,GrandTheftAutoV
97,1576423854,fox_score,For some reason all my vehicles will go from t...,Cars not leaving second gear,GrandTheftAutoV
98,1576423338,TheUsernamechosem3,,Having a bad day? Watch this upbeat GTA V Comp...,GrandTheftAutoV


In [7]:
final_df.to_csv('./data/GrandTheftAutoV.csv', index=False)